
NER with Gazetteer
A gazetteer is a dictionary of lexicons indicating entity groups.
Exercise
Write the function recognize_ngram() that takes a sequence of tokens and a gazetteer and returns a list of entities where each entity is represented by a tuple consisting of the following 4 items:
Index of the beginning token (inclusive)
Index of the ending token (exclusive)
Text span representing the entity (e.g., "Emory University")
Set of named entity tags for the entity

In [5]:
from typing import Dict, List, Tuple, Set

def recognize_ngram(tokens: List[str], gazetteer: Dict[str, Set[str]]) -> List[Tuple[int, int, str, Set[str]]]:
    """
    :param tokens: a sequence of input tokens.
    :param gazetteer: a dictionary whose key is the text span of a named entity (e.g., "Emory University") and the value is the set of named entity tags for the entity.
    :return: a list of entities where each entity is represented by a tuple consisting of the following 4 items:
             - Index of the beginning token (inclusive)
             - Index of the ending token (exclusive)
             - Text span representing the entity (e.g., "Emory University")
             - Set of named entity tags for the entity
    """
    entities = []
    for i in range(len(tokens)):
        for j in range(i+1, len(tokens)+1):
            key = ' '.join(tokens[i:j])
            val = gazetteer.get(key, None)
            if val: entities.append((i,j,key,val))
    return entities

In [6]:
GAZETTEER = {
    'Jinho': {'PER'},
    'Jinho Choi': {'PER'},
    'Emory': {'PER', 'ORG'},
    'Emory University': {'ORG'},
    'United States': {'GPE'},
    'United States of America': {'GPE'},
}

In [7]:
text = 'Jinho Choi is a professor at Emory University in the United States of America'
tokens = text.split()

entities = recognize_ngram(tokens, GAZETTEER)
for entity in entities: print(entity)

(0, 1, 'Jinho', {'PER'})
(0, 2, 'Jinho Choi', {'PER'})
(6, 7, 'Emory', {'PER', 'ORG'})
(6, 8, 'Emory University', {'ORG'})
(10, 12, 'United States', {'GPE'})
(10, 14, 'United States of America', {'GPE'})



NER using Prefix Tree
The recgonize_ngram() function requires $O(n^2)$ search that can be very slow. In this case, using a more advanced data structure such as Trie (aka. prefix tree) can significantly reduce the search complexity. In the following example, we will use the Aho–Corasick Algorithm that is based on Trie but more efficient in saving millions of entries.
Let us define a new gazetteer:

In [8]:
GAZETTEER = [
    ('Jinho', 'PER'),
    ('Jinho Choi', 'PER'),
    ('Emory', 'PER'),
    ('Emory', 'ORG'),
    ('Emory University', 'ORG'),
    ('United States', 'GPE'),
    ('United States of America', 'GPE'),
    ('Korean', 'LANG'),
    ('Korea', 'GPE'),
    ('South Korea', 'GPE'),
]

In [9]:
!pip install pyahocorasick
import ahocorasick

You should consider upgrading via the '/Users/carol/PycharmProjects/cs329/venv/bin/python -m pip install --upgrade pip' command.


pyahocorasick

We then write the create_ac() function to create the Aho-Corasick automation and add all entries in the gazetteer:

In [10]:
from typing import Iterable, Tuple, Any
from types import SimpleNamespace

def create_ac(data: Iterable[Tuple[str, Any]]) -> ahocorasick.Automaton:
    """
    Creates the Aho-Corasick automation and adds all (span, value) pairs in the data and finalizes this matcher.
    :param data: a collection of (span, value) pairs.
    """
    AC = ahocorasick.Automaton(ahocorasick.STORE_ANY)

    for span, value in data:
        if span in AC:
            t = AC.get(span)
        else:
            t = SimpleNamespace(span=span, values=set())
            AC.add_word(span, t)
        t.values.add(value)

    AC.make_automaton()
    return AC

In [11]:

a = {'A':0, 'B':1, 'C':2}
b = SimpleNamespace(A=0, B=1, C=2)

print(a['A'])
print(b.A)

0
0


In [12]:
AC = create_ac(GAZETTEER)

text = 'Dr. Choi is a Korean living in the United States of America '
tokens = text.split()

for eidx, t in AC.iter(text):
    print(eidx, t)

18 namespace(span='Korea', values={'GPE'})
19 namespace(span='Korean', values={'LANG'})
47 namespace(span='United States', values={'GPE'})
58 namespace(span='United States of America', values={'GPE'})


Let us write the match() function that returns a list of entities where each entity spans over only token boundaries:

In [13]:
from typing import List, Set

def match(AC: ahocorasick.Automaton, tokens: List[str]) -> List[Tuple[str, int, int, Set[str]]]:
    """
    :param AC: the finalized Aho-Corasick automation.
    :param tokens: the list of input tokens.
    :return: a list of tuples where each tuple consists of
             - span: str,
             - start token index (inclusive): int
             - end token index (exclusive): int
             - a set of values for the span: Set[str]
    """
    smap, emap, idx = dict(), dict(), 0
    for i, token in enumerate(tokens):
        smap[idx] = i
        idx += len(token)
        emap[idx] = i
        idx += 1

    # find matches
    text = ' '.join(tokens)
    spans = []
    for eidx, t in AC.iter(text):
        eidx += 1
        sidx = eidx - len(t.span)
        sidx = smap.get(sidx, None)
        eidx = emap.get(eidx, None)
        if sidx is None or eidx is None: continue
        spans.append((t.span, sidx, eidx + 1, t.values))

    return spans

In [14]:
for span in match(AC, tokens):
    print(span)

('Korean', 4, 5, {'LANG'})
('United States', 8, 10, {'GPE'})
('United States of America', 8, 12, {'GPE'})


Write the function remove_subsets() that removes entities that are subsets of other entities:

In [15]:
def remove_subsets(entities: List[Tuple[str, int, int, Set[str]]]) -> List[Tuple[str, int, int, Set[str]]]:
    """
    :param entities: a list of tuples where each tuple consists of
             - span: str,
             - start token index (inclusive): int
             - end token index (exclusive): int
             - a set of values for the span: Set[str] 
    :return: a list of entities where each entity is represented by a tuple of (span, start index, end index, value set)
    """
    tmp = []
    # TODO: To be updated
    for e0 in entities:
        remove = False
        for e1 in entities:
            if e0 == e1: continue
            if e0[1] >= e1[1] and e0[2] <= e1[2]:
                remove = True
                break
        if not remove: tmp.append(e0)
    return tmp

In [16]:
entities = match(AC, tokens)
entities = remove_subsets(entities)
for entity in entities: print(entity)

('Korean', 4, 5, {'LANG'})
('United States of America', 8, 12, {'GPE'})


NER with Large Gazetteers

In [17]:
import glob, os

def read_gazetteers(dirname: str) -> ahocorasick.Automaton:
    data = []
    for filename in glob.glob(os.path.join(dirname, '*.txt')):
        label = os.path.basename(filename)[:-4]
        for line in open(filename):
            data.append((line.strip(), label))
    return create_ac(data)

In [ ]:
AC = read_gazetteers('../dat/ner')

In [ ]:
for span in match(AC, tokens):
    print(span)